# Submit Predictions

This notebook shows how to prepare a submission of your model's predictions on the test data for the computation.

In [1]:
import os
import sys
sys.path.insert(1, os.path.realpath(os.path.pardir))

from pathlib import Path
from natsort import natsorted
import numpy as np
from safetensors.torch import load_model
import torch 
import pandas as pd

from utils import hvatnet
from utils.creating_dataset import LEFT_TO_RIGHT_HAND


## Load pre-trained model

This code loads the pre-trained baseline model - might be different for your model.

In [2]:
device = 'cuda:0'
dtype = torch.float32

# weights = r"C:\Users\feder\Documents\github\BCI_ALVI_challenge\tutorials\logs\test_2_run_fedya\step_3300_loss_0.2750.safetensors"
weights = "/media/lutetia/Extreme SSD/EMG_Yun/BCI_ALVI_challenge/tutorials/logs/test_2_run_fedya/step_6000_loss_0.1432.safetensors"

MODEL_TYPE = 'hvatnet'
model_config = hvatnet.Config(n_electrodes=8, n_channels_out=20,
                            n_res_blocks=3, n_blocks_per_layer=3,
                            n_filters=128, kernel_size=3,
                            strides=(2, 2, 2), dilation=2, 
                            small_strides = (2, 2))



# model_config = hvatnet.Config(n_electrodes=8, n_channels_out=20,
#                             n_res_blocks=2, n_blocks_per_layer=5,
#                             n_filters=82, kernel_size=6,
#                             strides=(2, 2, 2), dilation=3, 
#                             small_strides = (2, 2))

model = hvatnet.HVATNetv3(model_config)

load_model(model, weights)
# torch.load(weights, map_location=device)

model = model.to(device).to(dtype)

Number of parameters: 4210788


### Save `submission.cvs` file

This code shows how the data was prepare and downsampled during inference. Make sure that your data is processed similarly!

In [3]:
# DATA_PATH = Path(r"F:\Dropbox (Personal)\BCII\BCI Challenges\2024 ALVI EMG Decoding\dataset_v2_blocks\dataset_v2_blocks")
DATA_PATH = Path("/media/lutetia/Extreme SSD/EMG_Yun/bci-initiative-alvi-hci-challenge/dataset_v2_blocks/dataset_v2_blocks")

test_data_name = 'fedya_tropin_standart_elbow_left'  # shoould match `test_dataset_list` used to train the model


data_folder = DATA_PATH / "amputant" / "left" / test_data_name / "preproc_angles" / "submit"
all_paths = natsorted(data_folder.glob('*.npz'))
print(f'Found {len(all_paths)} samples in {data_folder}')


Found 72 samples in /media/lutetia/Extreme SSD/EMG_Yun/bci-initiative-alvi-hci-challenge/dataset_v2_blocks/dataset_v2_blocks/amputant/left/fedya_tropin_standart_elbow_left/preproc_angles/submit


In [4]:

pred_list = []

# loop over each trial
for i, p in enumerate(all_paths):
    # get EMG data 
    sample = np.load(p)
    myo = sample['data_myo']
    myo = myo[:, LEFT_TO_RIGHT_HAND]

    # predictions will have to be downsampled
    gt_len = myo[::8].shape[0]

    # padding
    target_length = (myo.shape[0] + 255) // 256 * 256
    padded_myo = np.pad(myo, ((0, target_length - myo.shape[0]), (0, 0)), mode='constant', constant_values=0)

    # some prediction. might be slididng window.
    preds = model.inference(padded_myo)
    preds_downsampled = preds[:gt_len]
    print(f"Completed {i+1}/{len(all_paths)}. Loaded data: {myo.shape} - padded to: {padded_myo.shape} - predictions {preds.shape} - downsampled to: {preds_downsampled.shape}")
    pred_list.append(preds_downsampled)

pred_cat = np.concatenate(pred_list, axis=0)
df = pd.DataFrame(pred_cat)
df.head()

Completed 1/72. Loaded data: (3721, 8) - padded to: (3840, 8) - predictions (480, 20) - downsampled to: (466, 20)
Completed 2/72. Loaded data: (3725, 8) - padded to: (3840, 8) - predictions (480, 20) - downsampled to: (466, 20)
Completed 3/72. Loaded data: (3724, 8) - padded to: (3840, 8) - predictions (480, 20) - downsampled to: (466, 20)
Completed 4/72. Loaded data: (3724, 8) - padded to: (3840, 8) - predictions (480, 20) - downsampled to: (466, 20)
Completed 5/72. Loaded data: (3729, 8) - padded to: (3840, 8) - predictions (480, 20) - downsampled to: (467, 20)
Completed 6/72. Loaded data: (3726, 8) - padded to: (3840, 8) - predictions (480, 20) - downsampled to: (466, 20)
Completed 7/72. Loaded data: (3724, 8) - padded to: (3840, 8) - predictions (480, 20) - downsampled to: (466, 20)
Completed 8/72. Loaded data: (3728, 8) - padded to: (3840, 8) - predictions (480, 20) - downsampled to: (466, 20)
Completed 9/72. Loaded data: (3724, 8) - padded to: (3840, 8) - predictions (480, 20) - 

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,0.089955,-0.060439,0.571276,0.205015,0.111516,-0.006313,0.596547,0.191327,0.143408,0.012984,0.625772,0.361150,0.141376,0.026459,0.507977,0.235472,0.040009,-0.004001,0.422167,0.074512
1,0.103219,-0.050257,0.572428,0.207906,0.122153,-0.009480,0.577139,0.192901,0.136306,0.015717,0.647213,0.360150,0.151859,0.023004,0.474166,0.201192,0.032529,0.011021,0.423579,0.077516
2,0.094456,-0.051845,0.577913,0.202574,0.115665,-0.005140,0.577403,0.174287,0.138520,0.015644,0.650071,0.354004,0.167817,0.026320,0.485393,0.217547,0.043441,-0.000750,0.421301,0.067791
3,0.103531,-0.073677,0.554439,0.194511,0.121958,-0.008796,0.571089,0.171539,0.164818,0.016437,0.615801,0.342139,0.182797,0.038546,0.480850,0.222084,0.049673,-0.008141,0.415423,0.053391
4,0.107211,-0.067394,0.526459,0.174771,0.127038,-0.004176,0.545070,0.143985,0.158092,0.017035,0.616378,0.345608,0.191944,0.035174,0.472926,0.193653,0.048328,-0.013225,0.412460,0.046231


In addition to the predictions, your data should also include a sample id column.

In [5]:
df.insert(0, "sample_id", range(1, 1 + len(df)))

Finally, save to a CSV file. This is what you'll upload to Kaggle for the competition.

In [6]:
df.to_csv('submit_file_Wave_MSE_Att_ver2.csv', index=False)

In [7]:
df.shape

(26829, 21)